# The Regular usage of collie

In [ ]:
import sys
import os
sys.path.append("../..")

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

# from collie import (
#     Transformer,
#     Trainer,
#     Evaluator,
#     Pusher,
#     Event,
#     TrainerPayload,
#     TransformerPayload,
#     EvaluatorPayload,
#     PusherPayload,
#     Orchestrator
# )
from collie.core.transform.transform import Transformer
from collie.core.tuner.tuner import Tuner
from collie.core.trainer.trainer import Trainer
from collie.core.evaluator.evaluator import Evaluator
from collie.core.pusher.pusher import Pusher
from collie.core.orchestrator.orchestrator import Orchestrator
from collie.contracts.event import Event
from collie.core.models import (
    TransformerPayload,
    TrainerPayload,
    TunerPayload,
    EvaluatorPayload,
    PusherPayload,
    TrainerArtifact,
    TransformerArtifact,
    TunerArtifact,
    EvaluatorArtifact,
    PusherArtifact
)

In [ ]:
num_samples = 1000
input_dim = 20   
num_classes = 4

## Transformer

In [ ]:
class MLPTransformer(Transformer):
    def __init__(self) -> None:
        super().__init__()

    def handle(self, event) -> Event:

        X = torch.randn(num_samples, input_dim)
        y = torch.randint(0, num_classes, (num_samples,))

        X_data = pd.DataFrame(X.numpy(), columns=[f"feature_{i}" for i in range(input_dim)])
        y_data = pd.DataFrame(y.numpy(), columns=["label"])

        train_data = pd.concat([X_data, y_data], axis=1)

        return Event(
            payload=TransformerPayload(
                train_data=train_data,
                validation_data=None,
                test_data=None
            )
        )

## Trainer

In [ ]:
class SimpleClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        return self.fc2(x)


class MLPTrainer(Trainer):
    def __init__(self):
        super().__init__()
        self.model = SimpleClassifier()
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def handle(self, event):
        
        train_data = event.payload.train_data

        X = torch.tensor(train_data.drop("label", axis=1).values, dtype=torch.float32)
        y = torch.tensor(train_data["label"].values, dtype=torch.long) 

        dataset = TensorDataset(X, y)
        dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
        epochs = 10
        for epoch in range(1, epochs + 1):
            self.model.train()
            total_loss = 0.0
            for xb, yb in dataloader:
                xb, yb = xb.to(self.device), yb.to(self.device)
                self.optimizer.zero_grad()
                logits = self.model(xb)
                loss = self.criterion(logits, yb)
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()

            self.log_metric("learning rate", self.scheduler.get_last_lr()[0], step=epoch)
            self.log_metric("loss", round(total_loss/len(dataloader), 3), step=epoch)
            
        return Event(
            payload=TrainerPayload(
                model=self.model,
                train_loss=total_loss/len(dataloader),
                val_loss=None
            )
        )

## Evaluator

In [ ]:
class MLPEvaluator(Evaluator):
    def __init__(
        self,
        registered_model_name="MLPClassifier",
        model_uri="" 
    ) -> None:
        super().__init__(
            registered_model_name=registered_model_name, 
            model_uri=model_uri
        )

    def handle(self, event):
        model = event.payload.model
        train_loss = event.payload.train_loss
        #mock the production metrics
        production_metric = 10

        return Event(
            payload=EvaluatorPayload(
                metrics={"Experiment": train_loss, "Production": production_metric},
                greater_is_better=False
            )
        )

## Pusher

In [ ]:
class MLPPusher(Pusher):
    def __init__(
        self,
        registered_model_name="MLPClassifier"
    ) -> None:
        super().__init__(registered_model_name=registered_model_name)

    def handle(self, event):
        return Event(
            payload=PusherPayload(
                model_uri="mlp_model_uri",
            )
        )

## Main

In [ ]:
orchestrator = Orchestrator(
    tracking_uri="http://localhost:5001",
    components=[
        MLPTransformer(),
        MLPTrainer(),
        MLPEvaluator(),
        MLPPusher()
    ],
    mlflow_tags={"Example": "MLP"},
    experiment_name="MLP2",
)
orchestrator.run()

2025/09/25 23:09:27 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/09/25 23:09:27 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
/Users/apple/opt/anaconda3/envs/collie/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '/Users/apple/Documents/PythonProject/collie/example/mlp/mlruns/1/c8cc802e8a7a45699b18af69a808132e/artifacts/train.csv'. Exception: 
  return _dataset_source_registry.resolve(
/Users/apple/opt/anaconda3/envs/collie/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/Users/a

🏃 View run Transformer at: http://localhost:5001/#/experiments/1/runs/c8cc802e8a7a45699b18af69a808132e
🧪 View experiment at: http://localhost:5001/#/experiments/1


2025/09/25 23:09:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/25 23:09:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/25 23:09:35 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/09/25 23:09:35 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
2025/09/25 23:09:35 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/09/25 23:09:35 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
Registered model 'MLPClassifier' already exists. Creating a new version of this model...
2025/09/25 23:09:35 WARNING mlflow.tracking._model_registry.fluent: Run with id bda6a7685c8a47698c70b1652142f4df has no artifacts 

🏃 View run Trainer at: http://localhost:5001/#/experiments/1/runs/bda6a7685c8a47698c70b1652142f4df
🧪 View experiment at: http://localhost:5001/#/experiments/1
Model URI: runs:/bda6a7685c8a47698c70b1652142f4df/model


Created version '3' of model 'MLPClassifier'.
/Users/apple/Documents/PythonProject/collie/example/mlp/../../collie/contracts/mlflow.py:373: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = self.mlflow_client.get_latest_versions(
/Users/apple/Documents/PythonProject/collie/example/mlp/../../collie/contracts/mlflow.py:256: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  self.mlflow_client.transition

🏃 View run Evaluator at: http://localhost:5001/#/experiments/1/runs/46106631b8684885a84d18a607f3286f
🧪 View experiment at: http://localhost:5001/#/experiments/1
🏃 View run Pusher at: http://localhost:5001/#/experiments/1/runs/0c720dc099a341a5a45cd71ddc1f0172
🧪 View experiment at: http://localhost:5001/#/experiments/1
🏃 View run Orchestrator at: http://localhost:5001/#/experiments/1/runs/6510b2de246e4655b92d6756d535209c
🧪 View experiment at: http://localhost:5001/#/experiments/1


In [ ]:
# TODO:
# TEST pusher tuner